## Chapter 24: Continuous Integration and Deployment (CI/CD)

Building and deploying applications manually is error‑prone, slow, and inconsistent. **Continuous Integration (CI)** and **Continuous Deployment (CD)** automate the process of building, testing, and delivering your software. With CI/CD, every code change triggers an automated pipeline that ensures the application is always in a deployable state. In this chapter, you’ll learn the principles of CI/CD, how to set up pipelines using **GitHub Actions** (a popular, free option for public repositories) and **Azure DevOps**, and how to deploy your containerized ASP.NET Core application to cloud platforms like **Azure App Service** and **Kubernetes**. By the end, you’ll be able to automate your entire delivery process, from commit to production.

### 24.1 Introduction to CI/CD

**Continuous Integration (CI)** is the practice of merging all developer working copies to a shared mainline several times a day. Each merge triggers an automated build and test process. The goal is to detect integration errors as quickly as possible.

**Continuous Delivery (CD)** extends CI by automatically deploying the application to a staging environment after the build and tests pass. With manual approval, you can then deploy to production.

**Continuous Deployment** takes it a step further: every change that passes all stages of the production pipeline is released to customers automatically, with no human intervention.

#### Key Benefits

- **Faster time to market**: Automate repetitive steps.
- **Higher quality**: Tests run automatically on every change.
- **Reduced risk**: Small, frequent deployments are easier to troubleshoot and roll back.
- **Consistency**: The deployment process is standardized and repeatable.

#### CI/CD Pipeline Stages

A typical pipeline consists of:

1. **Source**: Triggered by a push to a repository (GitHub, GitLab, Azure Repos).
2. **Build**: Compile code, restore dependencies, run unit tests.
3. **Test**: Run integration tests, code analysis, security scans.
4. **Package**: Create a deployable artifact (e.g., a Docker image, a zip file).
5. **Deploy**: Push the artifact to a target environment (dev, staging, production).

### 24.2 CI/CD with GitHub Actions

GitHub Actions is a powerful, integrated CI/CD platform that lives right inside your GitHub repository. It uses YAML‑based workflows that define steps to run on GitHub’s hosted runners (or your own).

#### Creating a Workflow

In your repository, create the directory `.github/workflows` and add a file, e.g., `ci-cd.yml`.

**Example: CI Pipeline for ASP.NET Core**

```yaml
name: CI Build and Test

on:
  push:
    branches: [ main, develop ]
  pull_request:
    branches: [ main ]

jobs:
  build:
    runs-on: ubuntu-latest

    steps:
    - uses: actions/checkout@v4

    - name: Setup .NET
      uses: actions/setup-dotnet@v4
      with:
        dotnet-version: 8.0.x

    - name: Restore dependencies
      run: dotnet restore

    - name: Build
      run: dotnet build --no-restore --configuration Release

    - name: Test
      run: dotnet test --no-build --configuration Release --verbosity normal

    - name: Publish
      run: dotnet publish -c Release -o ./publish

    - name: Upload artifact
      uses: actions/upload-artifact@v4
      with:
        name: webapp
        path: ./publish
```

This workflow:
- Triggers on pushes to `main`/`develop` and pull requests to `main`.
- Checks out code, sets up .NET 8, restores dependencies, builds, tests, publishes, and uploads the build output as an artifact for later use.

#### Adding Deployment to Azure App Service (Linux)

To deploy, you need to add a deployment job. First, set up an Azure service principal and store its credentials as secrets in your GitHub repository (e.g., `AZURE_WEBAPP_PUBLISH_PROFILE` or `AZURE_CREDENTIALS`).

```yaml
deploy:
  runs-on: ubuntu-latest
  needs: build
  if: github.ref == 'refs/heads/main'  # only deploy from main

  steps:
    - name: Download artifact
      uses: actions/download-artifact@v4
      with:
        name: webapp
        path: ./publish

    - name: Deploy to Azure Web App
      uses: azure/webapps-deploy@v2
      with:
        app-name: 'my-aspnet-app'
        publish-profile: ${{ secrets.AZURE_WEBAPP_PUBLISH_PROFILE }}
        package: ./publish
```

#### Deploying a Docker Image to Azure Container Registry and App Service

If you’re using Docker, you can build and push an image to a registry, then deploy.

```yaml
jobs:
  build-and-push:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4

      - name: Login to Azure Container Registry
        uses: azure/docker-login@v1
        with:
          login-server: myregistry.azurecr.io
          username: ${{ secrets.REGISTRY_USERNAME }}
          password: ${{ secrets.REGISTRY_PASSWORD }}

      - name: Build and push Docker image
        run: |
          docker build -t myregistry.azurecr.io/myapp:${{ github.sha }} .
          docker push myregistry.azurecr.io/myapp:${{ github.sha }}

      - name: Deploy to Azure Web App for Containers
        uses: azure/webapps-deploy@v2
        with:
          app-name: 'my-container-app'
          images: 'myregistry.azurecr.io/myapp:${{ github.sha }}'
```

### 24.3 CI/CD with Azure DevOps

Azure DevOps provides a full suite of development tools, including pipelines. You define pipelines in YAML (similar to GitHub Actions) or using the classic visual designer.

#### Setting Up a Pipeline

Create an `azure-pipelines.yml` in your repository.

```yaml
trigger:
- main
- develop

pool:
  vmImage: 'ubuntu-latest'

variables:
  buildConfiguration: 'Release'

steps:
- task: UseDotNet@2
  inputs:
    packageType: 'sdk'
    version: '8.x'

- task: DotNetCoreCLI@2
  displayName: 'Restore packages'
  inputs:
    command: 'restore'

- task: DotNetCoreCLI@2
  displayName: 'Build'
  inputs:
    command: 'build'
    arguments: '--configuration $(buildConfiguration) --no-restore'

- task: DotNetCoreCLI@2
  displayName: 'Run tests'
  inputs:
    command: 'test'
    arguments: '--configuration $(buildConfiguration) --no-build'

- task: DotNetCoreCLI@2
  displayName: 'Publish'
  inputs:
    command: 'publish'
    publishWebProjects: true
    arguments: '--configuration $(buildConfiguration) --output $(Build.ArtifactStagingDirectory)'

- task: PublishBuildArtifacts@1
  inputs:
    PathtoPublish: '$(Build.ArtifactStagingDirectory)'
    ArtifactName: 'drop'
```

#### Deploy to Azure App Service

After the build, you can add a release stage in Azure DevOps (using Releases) or extend the YAML pipeline with deployment jobs.

```yaml
- stage: Deploy
  jobs:
  - deployment: DeployToAzure
    environment: 'production'
    strategy:
      runOnce:
        deploy:
          steps:
          - task: AzureWebApp@1
            inputs:
              azureSubscription: 'your-service-connection'
              appName: 'my-aspnet-app'
              package: '$(Pipeline.Workspace)/drop/**/*.zip'
```

### 24.4 Building a CI/CD Pipeline Step by Step

Let’s walk through a complete example using GitHub Actions for a containerized ASP.NET Core application with a database.

#### Step 1: Prepare the Repository

Ensure your solution has:
- A `Dockerfile` for the web project.
- Unit and integration tests that can run in a headless environment.
- (Optional) A `docker-compose.yml` for testing multi‑container scenarios.

#### Step 2: Define the Workflow

Create `.github/workflows/docker-ci-cd.yml`:

```yaml
name: Build, Test, and Deploy

on:
  push:
    branches: [ main ]
  pull_request:
    branches: [ main ]

env:
  REGISTRY: ghcr.io
  IMAGE_NAME: ${{ github.repository }}

jobs:
  build-and-test:
    runs-on: ubuntu-latest

    services:
      sqlserver:
        image: mcr.microsoft.com/mssql/server:2022-latest
        env:
          SA_PASSWORD: "YourStrong!Password"
          ACCEPT_EULA: "Y"
        ports:
          - 1433:1433
        options: >-
          --health-cmd "sqlcmd -S localhost -U sa -P 'YourStrong!Password' -Q 'SELECT 1'"
          --health-interval 10s
          --health-timeout 5s
          --health-retries 5

    steps:
      - uses: actions/checkout@v4

      - name: Setup .NET
        uses: actions/setup-dotnet@v4
        with:
          dotnet-version: 8.0.x

      - name: Restore
        run: dotnet restore

      - name: Build
        run: dotnet build --configuration Release --no-restore

      - name: Test
        env:
          ConnectionStrings__DefaultConnection: "Server=localhost;Database=MyAppDb;User Id=sa;Password=YourStrong!Password;TrustServerCertificate=true"
        run: dotnet test --configuration Release --no-build

  docker-build-push:
    needs: build-and-test
    runs-on: ubuntu-latest
    if: github.ref == 'refs/heads/main' && github.event_name == 'push'

    steps:
      - uses: actions/checkout@v4

      - name: Log in to GitHub Container Registry
        uses: docker/login-action@v3
        with:
          registry: ${{ env.REGISTRY }}
          username: ${{ github.actor }}
          password: ${{ secrets.GITHUB_TOKEN }}

      - name: Build and push Docker image
        uses: docker/build-push-action@v5
        with:
          context: .
          push: true
          tags: ${{ env.REGISTRY }}/${{ env.IMAGE_NAME }}:${{ github.sha }}, ${{ env.REGISTRY }}/${{ env.IMAGE_NAME }}:latest

  deploy:
    needs: docker-build-push
    runs-on: ubuntu-latest
    environment: production

    steps:
      - name: Deploy to Azure Web App for Containers
        uses: azure/webapps-deploy@v2
        with:
          app-name: 'my-aspnet-app'
          publish-profile: ${{ secrets.AZURE_WEBAPP_PUBLISH_PROFILE }}
          images: '${{ env.REGISTRY }}/${{ env.IMAGE_NAME }}:${{ github.sha }}'
```

#### Step 3: Set Secrets

Add the following secrets to your GitHub repository:
- `GITHUB_TOKEN` is automatically provided.
- `AZURE_WEBAPP_PUBLISH_PROFILE`: Download the publish profile from Azure App Service and paste it as a secret.

### 24.5 Deploying to Kubernetes

If you’re using Kubernetes, you can add a step that updates the image tag in a deployment manifest and applies it to the cluster.

```yaml
deploy:
  needs: docker-build-push
  runs-on: ubuntu-latest
  steps:
    - uses: actions/checkout@v4

    - name: Set up kubectl
      uses: azure/setup-kubectl@v3
      with:
        version: 'latest'

    - name: Set AKS context
      uses: azure/aks-set-context@v3
      with:
        resource-group: 'myResourceGroup'
        cluster-name: 'myAKSCluster'
        credentials: ${{ secrets.AZURE_CREDENTIALS }}

    - name: Update image in deployment
      run: |
        kubectl set image deployment/myapp myapp=${{ env.REGISTRY }}/${{ env.IMAGE_NAME }}:${{ github.sha }} -n production
        kubectl rollout status deployment/myapp -n production
```

### 24.6 Secrets Management

Never hardcode secrets in your pipeline files. Use the built‑in secrets features:

- **GitHub Actions**: Add secrets via repository settings → Secrets and variables → Actions. Access with `${{ secrets.SECRET_NAME }}`.
- **Azure DevOps**: Use variable groups or Azure Key Vault integration.
- For Kubernetes, consider using tools like `Sealed Secrets` or external secrets operators.

### 24.7 Testing in CI/CD

Your pipeline should run tests. For unit tests, `dotnet test` works. For integration tests that require a database, you can spin up dependent services as containers (as shown earlier with `services` in GitHub Actions). This gives you a clean, isolated test environment for each run.

### 24.8 Best Practices

- **Fail fast**: Run the fastest tests first.
- **Idempotent pipelines**: Running the same commit twice should produce the same result.
- **Immutable artifacts**: Build once, deploy many times. Use the same artifact (or Docker image) in all environments.
- **Versioning**: Tag your builds and Docker images with the commit hash or semantic version.
- **Environment parity**: Keep development, staging, and production as similar as possible.
- **Secure secrets**: Use secret stores, not plain text.
- **Monitor pipeline health**: Set up notifications for failures.

### 24.9 Summary

In this chapter, you’ve learned how to automate the build, test, and deployment of your ASP.NET Core applications using CI/CD pipelines:

- **CI/CD principles** and benefits.
- **GitHub Actions** for building, testing, and deploying to Azure App Service.
- **Azure DevOps** as an alternative.
- **Docker integration** to build and push images to a registry.
- **Deploying to Kubernetes**.
- **Secrets management** and best practices.

With CI/CD, you can ship features faster and with greater confidence. Automating your pipeline is a key step toward professional, reliable software delivery.

**Exercise:**

1. Set up a GitHub repository for your ASP.NET Core project (or use an existing one).
2. Create a GitHub Actions workflow that restores, builds, and runs tests on every push.
3. If you have an Azure account, deploy your application to Azure App Service using the workflow (you may need to set up a free trial).
4. Add a step that builds a Docker image and pushes it to GitHub Container Registry (GHCR).
5. (Optional) Create a simple Kubernetes deployment and set up a workflow to update the image.

In the next chapter, **"Monitoring and Observability,"** you’ll learn how to gain insight into your running applications using logs, metrics, distributed tracing, and health checks. This will help you detect issues, understand performance, and maintain reliability in production.